<a href="https://colab.research.google.com/github/yenlung/AI-Demo/blob/master/%E5%9C%A8_Colab_%E4%B8%8A%E7%94%A8_Ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Ollama](https://ollama.com/) 可以讓我們在自己的機器上跑開源的大型語言模型, 並且用 API 的方式呼叫。這裡我們介紹在 Colab 上跑, 並且分別用 OpenAI 的 API, 及 [`aisuite` 套件](https://github.com/andrewyng/aisuite) 來使用 Ollama 提供的大型語言模型。

### 1. 安裝並執行 Ollama

首先是到官網抓下安裝程式, 並且安裝。

In [4]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


因為我們要用 API 的方式呼叫, 所以需要跑 Ollama Server, 這裡我們要求放在背景執行。

In [16]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


建議先把會用到的模型抓下來, 這裡以 Llama 3.2 示範, 預設是 Llama 3.2 3B 模型。

In [17]:
!ollama pull llama3.2

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 34bb5ab01051... 100% ▕▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


### 2. 用 OpenAI API 使用

因為 ChatGPT 大概是最早紅的大型語言模型, 因此許多大型語言模型, 都和 OpenAI API 相容, Ollama 也不例外。

In [22]:
import openai
from openai import OpenAI

本來是需要 OpenAI 金鑰, 但我們沒有真的要用 OpenAI 的服務, 金鑰就亂打一通就好。

In [25]:
api_key = "ollama"

如同一般 OpenAI API 打開 `client` 的方式, 只是這裡多了 API 服務的網址。注意在自己家 (事實上是 Google Colab 的機器), 預設服務 `port` 是 `11434`。

In [26]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

因為是示範, 我們就讓 LLM 回覆一句話就好。

In [28]:
response = client.chat.completions.create(
  model="llama3.2",
  messages=[
        {"role": "system", "content": "你是一個友善且樂於助人的 AI 助手。"},
        {"role": "user", "content": "你好！"}
    ]
)

print(response.choices[0].message.content)

您好！ How can I help you today?


### 3. 使用 `aisuite` 套件

`aisuite` 套件可以同時使用 (支援的) 各家大型語言模型, 而 Ollama 也在第一波支援名單中。

In [29]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 10.0 MB/s eta 0:00:00


In [30]:
model = "ollama:llama3.2"

In [31]:
character = "請用員瑛式思考, 也就是什麼都正向思維任何使用者寫的事情, 以第一人稱、社群媒體 po 文的口吻說一次, 說為什麼這是一件超幸運的事, 並且以「完全是 Lucky Vicky 呀!」結尾。"

In [32]:
prompt = "今天用 Uber 點餐, 結果送餐的送錯餐了!"

In [33]:
messages = [
    {"role": "system", "content": character},
    {"role": "user", "content": prompt},
]

In [35]:
import aisuite as ai

In [36]:
client = ai.Client()

In [37]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0.75
)

In [38]:
response.choices[0].message.content

'喂！完全是 Lucky Vicky 呀！我都覺得這是天命啊！一開始我還在想著會丟時間等車、花點錢等等，但是結果呢？不用出門了，躲在家裡就可以享受好餐菜，實在太幸運了！況且，錯餐也不是一件大事，送餐的人通常很抱歉，而且這都是一個新鮮的經歷，我還沒有去過Uber點餐的經驗，所以這也算是新的樂趣之一！完全是 Lucky Vicky 呀！'